In [1]:
import math
import os

import numpy as np

import pandas as pd

from scipy.interpolate import interp1d

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    import pysynphot as synphot

In [2]:
d1 = 40
d2 = 19
area = math.pi * (d1/2)**2 - math.pi * (d2/2)**2
print('OTA area: {:.0f} cm2'.format(area))

dark_current = 0.0010 # e-/s/pix
readout = 13.0 # e-/pix
plate_scale = 1.24 # arcsec/pix

airmass = 1.0
seeing = 1.5 # arcsec
binfac = 1
plate_scale = plate_scale * binfac
target_snr = 5
exptime = 60

OTA area: 973 cm2


In [3]:
data_dir = 'data'
wave_range = np.arange(300, 900, 1)

# Read in sky background data files, interpolate to the given wave range and store
sky_data = {}
for filename in sorted(os.listdir(data_dir)):
    if not filename.endswith('.data') or 'sky' not in filename:
        continue
    key = filename.split('.')[0]
    data = pd.read_csv(os.path.join(data_dir, filename), names=['wave', 'throughput'], index_col=0, comment='#', sep=' ')
    data_interp = interp1d(data.index, data.throughput, bounds_error=False, fill_value='extrapolate')(wave_range)
    sky_data[key] = data_interp

# Make into a dataframe
sky_data = pd.DataFrame(data=sky_data, index=wave_range)

# Save all data as a CSV
sky_data.to_csv(os.path.join(data_dir, 'sky.csv'))
#sky_data

In [4]:
print('| Filter | Zeropoint </br> (AB mag) | Zeropoint </br> (Vega mag) | Dark sky flux  | Grey sky flux | Bright sky flux | Extinction </br> (mag/airmass) | Limiting mag |')
print('| - | - | - | - | - | - | - | - |')

for filt in ['L', 'R', 'G', 'B']:
    print('', end = '|')

    print(f' {filt} ', end = '|')
    
    # Create a PySynphot bandpass of the full throughput model (no atmosphere)
    noatm_data = pd.read_csv(os.path.join(data_dir, f'GOTO_{filt}_noatm.data'), names=['wave', 'throughput'], index_col=0, comment='#', sep=' ')
    noatm_bandpass = synphot.ArrayBandpass(noatm_data.index*10, noatm_data['throughput'], waveunits='Angstrom')
    
    # Make an observation of a flat spectrum (AB mag zeropoint)
    flat_spec = synphot.FlatSpectrum(3631., fluxunits='Jy')
    flat_obs = synphot.Observation(flat_spec, noatm_bandpass, binset=noatm_bandpass.wave)
    flat_N = flat_obs.integrate('photlam') * area
    flat_zp = 0 - -2.5*np.log10(flat_N/1)
    print(f' {flat_zp:>4.2f} ', end = '|')
    
    # Make an observation of a Vega spectrum (Vega mag zeropoint)
    vega_spec = synphot.Vega
    vega_spec.convert('photlam')
    vega_obs = synphot.Observation(vega_spec, noatm_bandpass, binset=noatm_bandpass.wave)
    vega_N = vega_obs.integrate('photlam') * area
    vega_zp = 0 - -2.5*np.log10(vega_N/1)
    print(f' {vega_zp:>4.2f} ', end = '|')
    
    for phase in ['sky_dark', 'sky_grey', 'sky_bright']:
        # Make an observation of the sky spectrum
        sky_spec = synphot.ArraySpectrum(sky_data.index*10, sky_data[phase], waveunits='Angstrom', fluxunits='flam')
        sky_spec.convert('photlam')
        sky_obs = synphot.Observation(sky_spec, noatm_bandpass, binset=noatm_bandpass.wave)
        sky_flux = sky_obs.integrate('photlam')
        print(f' {sky_flux:>.4f} ', end = '|')
        
    # Create a PySynphot bandpass of the full throughput model (including atmosphere)
    atm_data = pd.read_csv(os.path.join(data_dir, f'GOTO_{filt}.data'), names=['wave', 'throughput'], index_col=0, comment='#', sep=' ')
    atm_bandpass = synphot.ArrayBandpass(atm_data.index*10, atm_data['throughput'], waveunits='Angstrom')
    
    # Get extinction by comparing atm and no_atm observations
    atm_obs = synphot.Observation(flat_spec, atm_bandpass, binset=atm_bandpass.wave)
    noatm_obs = synphot.Observation(flat_spec, noatm_bandpass, binset=noatm_bandpass.wave)
    extinction = atm_obs.integrate() / noatm_obs.integrate()
    extinction = -2.5*np.log10(extinction)
    print(f' {extinction:.3f} ', end = '|')
    
    # ~~~~~~~~~~~~~~~~~~~~~
    # Calculate number of pixels in seeing disc
    n_pixels = np.pi * ((1.5*seeing)/plate_scale)**2.
    
    # Calculate sky flux
    sky_spec = synphot.ArraySpectrum(sky_data.index*10, sky_data['sky_dark'], waveunits='Angstrom', fluxunits='flam')
    sky_spec.convert('photlam')
    sky_obs = synphot.Observation(sky_spec, noatm_bandpass, binset=noatm_bandpass.wave)
    sky_flux = sky_obs.integrate('photlam')
    
    # Calculate total noise
    sky_signal = sky_flux * area * plate_scale**2 # e-/s/pix
    sky_noise = sky_signal * exptime * n_pixels # e-
    dark_noise = dark_current * exptime * n_pixels # e-
    readout_noise = readout * n_pixels # e-
    total_noise = sky_noise + dark_noise + readout_noise**2 # e-

    # Calculate the object signal needed to get the target SNR
    # NB: found from inverting SNR equation & using quadratic form
    obj_signal = target_snr/2 * (target_snr + np.sqrt(target_snr**2 + 4*total_noise)) # e-/sec

    # Convert signal into instrumental magnitude
    lim_mag_inst = -2.5 * np.log10(obj_signal/exptime) # mag

    # Subtract atmospheric extinction
    lim_mag_inst = lim_mag_inst - extinction * airmass
    
    # Convert signal into observed magnitude using zeropoint
    lim_mag = lim_mag_inst + flat_zp
    
    print(f' {lim_mag:>4.2f} ', end = '|')

    print()    

| Filter | Zeropoint </br> (AB mag) | Zeropoint </br> (Vega mag) | Dark sky flux  | Grey sky flux | Bright sky flux | Extinction </br> (mag/airmass) | Limiting mag |
| - | - | - | - | - | - | - | - |
| L | 22.63 | 22.62 | 0.0031 | 0.0173 | 0.0331 | 0.148 | 19.76 |
| R | 21.33 | 21.14 | 0.0014 | 0.0051 | 0.0096 | 0.099 | 18.55 |
| G | 21.67 | 21.68 | 0.0012 | 0.0070 | 0.0139 | 0.144 | 18.85 |
| B | 21.66 | 21.78 | 0.0007 | 0.0070 | 0.0131 | 0.215 | 18.78 |
